In [1]:
import poppy     as poppy
import jax.numpy as jnp
from   jax       import random

In [8]:
p, n = 6661, 3
f = poppy.field( p, n )

In [9]:
d = 256
a = random.randint( poppy.KEY1, ( d, d ), 0, p ** n, dtype = jnp.int64 )
b = random.randint( poppy.KEY2, ( d, d ), 0, p ** n, dtype = jnp.int64 )

In [10]:
ap = poppy.array( a, f )
bp = poppy.array( b, f )

In [11]:
ap.shape, ap.rep.shape

((256, 256), (768, 768))

In [12]:
api = ap.inv( )
bpi = bp.inv( )

In [13]:
print( ap.rep[ 0, : 10 ] )
( ap - ap @ api @ ap @ bpi @ bp ).rep

[3916  633 5853 2182 1748  155  104 4188 3053 5265]


Array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)